In [146]:
import urllib
import pandas as pd
import numpy as np
from pandas import Series
import warnings
warnings.filterwarnings("ignore")
moviesRaw = 'https://raw.githubusercontent.com/dstern04/MSDA-Work/master/661%20-%20Recommender%20Systems/Assignment1/movies.csv'
ratingsRaw = 'https://raw.githubusercontent.com/dstern04/MSDA-Work/master/661%20-%20Recommender%20Systems/Assignment1/ratings.csv'

In [147]:
moviesCSV = urllib.urlopen(moviesRaw)
moviesDF = pd.read_csv(moviesCSV)
moviesDF.columns = ['movieId','title','genres']

ratingsCSV = urllib.urlopen(ratingsRaw)
ratingsDF = pd.read_csv(ratingsCSV)
ratingsDF.columns = ['userId','movieId','rating','timestamp']

Calculate, a movie's mean rating and return the top 10 movies by mean rating, with their mean, id number, and title.

In [148]:
meanRatings = ratingsDF.groupby(['movieId'],sort=True,as_index=False).mean()
meanRatings = pd.merge(meanRatings, moviesDF, on='movieId', how='left')
meanRatings[['rating','title','movieId']].sort(['rating'], ascending=[0]).head(10)

,rating,title,movieId
8360,5,Pageant (2008),93729
8272,5,Abduction (2011),90524
8213,5,Alpha and Omega (2010),88468
8214,5,Edison Kinetoscopic Record of a Sneeze (1894),88674
6077,5,Hearts and Minds (1974),26325
8220,5,30 Minutes or Less (2011),88812
8226,5,Miranda (1948),89018
6058,5,My Night At Maud's (Ma Nuit Chez Maud) (1969),26208
8237,5,"Interrupters, The (2011)",89337
8240,5,Restraint (2008),89367


Calculate a movie's number of ratings and return the 10 movies with the most ratings, with their number of ratings, id number, and title.

In [149]:
numRatings = pd.pivot_table(ratingsDF, values='rating', columns=['movieId'], aggfunc=len)
numRatings = numRatings.to_frame()
numRatings.reset_index(level=0, inplace=True)
numRatings.columns = ['movieId','numRatings']
numRatings = pd.merge(numRatings, moviesDF, on='movieId', how='left')
numRatings = numRatings[['numRatings','title','movieId']]
numRatings.sort(['numRatings'], ascending=[0]).head(10)

,numRatings,title,movieId
264,333,Pulp Fiction (1994),296
322,321,Forrest Gump (1994),356
285,310,"Shawshank Redemption, The (1994)",318
429,307,Jurassic Park (1993),480
526,299,"Silence of the Lambs, The (1991)",593
230,265,Star Wars: Episode IV - A New Hope (1977),260
0,263,Toy Story (1995),1
2130,247,"Matrix, The (1999)",2571
97,244,Braveheart (1995),110
523,244,Terminator 2: Judgment Day (1991),589


Calculate a movie's damped mean, with a damping term of 5. Return the top 10 movies by damped mean rating, with their damped mean, id number, and title. 

In [150]:
globalMean = ratingsDF['rating'].mean()
sumRatings = ratingsDF.groupby(['movieId'],sort=True,as_index=False).sum()
sumRatings = sumRatings[['movieId','rating']]
sumRatings = pd.merge(sumRatings, numRatings, on='movieId', how='left')
sumRatings['dampedMean'] = (sumRatings['rating'] + 5*globalMean)/(sumRatings['numRatings'] + 5)
sumRatings = sumRatings[['dampedMean','title','movieId']]
sumRatings.sort(['dampedMean'], ascending=[0]).head(10)

,dampedMean,title,movieId
285,4.442797,"Shawshank Redemption, The (1994)",318
708,4.429553,"Godfather, The (1972)",858
2306,4.404460,"Christmas Story, A (1983)",2804
47,4.399491,"Usual Suspects, The (1995)",50
1016,4.321967,Ran (1985),1217
1020,4.304368,"Godfather: Part II, The (1974)",1221
4866,4.290081,Ikiru (1952),6669
3304,4.284355,Hope and Glory (1987),4117
474,4.283534,Schindler's List (1993),527
1941,4.280061,Central Station (Central do Brasil) (1998),2357


Calculate the similarity of one movie to another based on how likely the user is to rate one given that they rated the other (ignoring the rating values), using the simple $\frac{x \wedge y}{x}$ method. Return the 10 movies most similar to movie 1389 (Jaws 3-D) using the simple metric, with their similarity scores, id number, and title.

In [151]:
allUsers = ratingsDF['userId'].unique()
movie1389ratings = ratingsDF.query('movieId==1389')
usersWhoRated1389 = movie1389ratings['userId'].unique()
ratingsDF['rated1389'] = 0
ratingsDF['didntRate1389'] = 0
for i in range(0,len(ratingsDF)):
    if ratingsDF.ix[i,'userId'] in usersWhoRated1389:
        ratingsDF.ix[i,'rated1389'] = 1
        ratingsDF.ix[i,'didntRate1389'] = 0
    else:
        ratingsDF.ix[i,'rated1389'] = 0
        ratingsDF.ix[i,'didntRate1389'] = 1
ratingsDF.head(5) # check output

,userId,movieId,rating,timestamp,rated1389,didntRate1389
0,1,1,5,847117005,0,1
1,1,2,3,847642142,0,1
2,1,10,3,847641896,0,1
3,1,32,4,847642008,0,1
4,1,34,4,847641956,0,1


In [152]:
sumXY = pd.pivot_table(ratingsDF, values=['rated1389','didntRate1389'], columns=['movieId'], aggfunc=sum)
sumXY = sumXY.transpose()
sumXY.reset_index(level=0, inplace=True)
sumXY['similarity1389'] = sumXY['rated1389']/len(usersWhoRated1389)
sumXY = pd.merge(sumXY, moviesDF, on='movieId', how='left')
sumXY = sumXY[['similarity1389','title','movieId']]
sumXY.sort(['similarity1389'], ascending=[0]).head(10)

,similarity1389,title,movieId
1181,1.000000,Jaws 3-D (1983),1389
429,0.866667,Jurassic Park (1993),480
526,0.866667,"Silence of the Lambs, The (1991)",593
523,0.866667,Terminator 2: Judgment Day (1991),589
647,0.866667,Independence Day (a.k.a. ID4) (1996),780
1009,0.866667,Star Wars: Episode VI - Return of the Jedi (1983),1210
996,0.866667,Star Wars: Episode V - The Empire Strikes Back...,1196
0,0.866667,Toy Story (1995),1
285,0.866667,"Shawshank Redemption, The (1994)",318
345,0.800000,True Lies (1994),380


Here we see that the most similar movie is Jaws 3D itself - a good indication that our formula worked.

Now we calculate the similarity of one movie to another, using the advanced $\frac{\frac{x \wedge y}{1+x}}{1+\frac{!x \wedge y}{1+!x}}$ method. We will return the 10 movies most similar to movie 1389 (Jaws 3-D) using the advanced metric, with their similarity scores, id number, and title.
 
First we need to recalculate sumXY with 1 in the denominator.

In [153]:
sumXY = pd.pivot_table(ratingsDF, values=['rated1389','didntRate1389'], columns=['movieId'], aggfunc=sum)
sumXY = sumXY.transpose()
sumXY.reset_index(level=0, inplace=True)
sumXY['similarity1389'] = sumXY['rated1389']/(1+len(usersWhoRated1389))
sumXY['dissimilarity1389'] = (1+sumXY['didntRate1389'])/(1+len(allUsers) - len(usersWhoRated1389))
sumXY['index'] = sumXY['similarity1389']/(1+sumXY['dissimilarity1389'])
sumXY = pd.merge(sumXY, moviesDF, on='movieId', how='left')
sumXY = sumXY[['index','title','movieId']]
sumXY.sort(['index'], ascending=[0]).head(11)

,index,title,movieId
1181,0.936170,Jaws 3-D (1983),1389
1180,0.675978,Jaws 2 (1978),1388
1021,0.664151,Full Metal Jacket (1987),1222
1641,0.658354,Back to the Future Part III (1990),2012
1179,0.658354,Jaws (1975),1387
1056,0.657534,Stand by Me (1986),1259
1063,0.641060,Unforgiven (1992),1266
315,0.630072,Clear and Present Danger (1994),349
860,0.629321,Die Hard (1988),1036
1037,0.629321,"Terminator, The (1984)",1240
